In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch 
from torch import nn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [3]:
train_df.shape

(42000, 785)

In [4]:
y = train_df["label"]
x = train_df.iloc[:,1:]

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, val , y, val_y =  train_test_split(x,y,test_size = 0.2 , random_state = 21)

In [7]:
train = torch.Tensor(train.values).reshape((33600, 1, 28, 28))

In [8]:
val = torch.Tensor(val.values).reshape((8400, 1, 28, 28))

In [9]:
y = torch.Tensor(y.values).reshape((33600, 1))

In [10]:
val_y = torch.Tensor(val_y.values).reshape((8400, 1))

In [11]:
y = y.squeeze()

In [12]:
val_y = val_y.squeeze()

In [13]:
class Digit(nn.Module):
    def __init__(self, num_classes=10):
        super(Digit, self).__init__()

        # Define the layers
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Adjusted for 28x28 input images
        self.fc2 = nn.Linear(128, 10)  # Assuming there are 10 classes 
        

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [14]:
model = Digit()

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

In [16]:
def acc(base,pred):
    correct = torch.eq(base,pred).sum().item()
    result = (correct/len(pred))*100
    return result

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:
epochs = 50

In [19]:
softmax = nn.Softmax(dim=1)
model.to(device)
train = train.to(device)
y = y.to(device)

for epoch in range(epochs):
    model.train()

    pred = model(train)
    loss = loss_fn(pred, y.long())

    _, preds = torch.max(softmax(pred), dim=1)
    accuracy = acc(y, preds)  

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val = val.to(device)    # Assuming you have a validation set
        val_y = val_y.to(device) # Assuming you have the corresponding labels

        pred_val = model(val)

        val_loss = loss_fn(pred_val, val_y.long())

        _, preds_val = torch.max(softmax(pred_val), dim=1)
        val_acc = acc(val_y, preds_val)

    if epoch % 1 == 0:
        print(f"Epoch: {epoch} Loss: {loss:.5f}, Accuracy: {accuracy:.2f}% Val_Loss: {val_loss:.5f}, Val_Accuracy: {val_acc:.2f}%")


Epoch: 0 Loss: 2.38891, Accuracy: 9.10% Val_Loss: 24.52322, Val_Accuracy: 9.86%
Epoch: 1 Loss: 12.05821, Accuracy: 10.49% Val_Loss: 20.68431, Val_Accuracy: 27.08%
Epoch: 2 Loss: 9.33755, Accuracy: 16.18% Val_Loss: 23.63474, Val_Accuracy: 14.54%
Epoch: 3 Loss: 8.36209, Accuracy: 19.79% Val_Loss: 20.27461, Val_Accuracy: 37.48%
Epoch: 4 Loss: 7.45393, Accuracy: 39.51% Val_Loss: 16.98113, Val_Accuracy: 40.50%
Epoch: 5 Loss: 7.21215, Accuracy: 34.60% Val_Loss: 15.17390, Val_Accuracy: 37.08%
Epoch: 6 Loss: 6.63679, Accuracy: 34.13% Val_Loss: 12.24157, Val_Accuracy: 46.63%
Epoch: 7 Loss: 5.75848, Accuracy: 45.13% Val_Loss: 10.27549, Val_Accuracy: 42.10%
Epoch: 8 Loss: 5.29118, Accuracy: 37.01% Val_Loss: 6.82859, Val_Accuracy: 50.90%
Epoch: 9 Loss: 3.48294, Accuracy: 51.01% Val_Loss: 4.42089, Val_Accuracy: 62.58%
Epoch: 10 Loss: 2.31899, Accuracy: 68.18% Val_Loss: 3.54996, Val_Accuracy: 62.19%
Epoch: 11 Loss: 2.31688, Accuracy: 58.65% Val_Loss: 3.33437, Val_Accuracy: 56.74%
Epoch: 12 Loss: 2.2

In [20]:
test_df = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
test_df.shape

(28000, 784)

In [21]:
tester = torch.Tensor(test_df.values).reshape((28000, 1, 28, 28))

In [22]:
model.eval()
pred = model(tester)

In [23]:
_, preds_val = torch.max(softmax(pred), dim=1)

In [24]:
result = pd.DataFrame(preds_val).to_csv("submission.csv")